# About

These is a base solution of PID.

In [2]:
%matplotlib inline
import random
import pandas
import numpy
import cPickle as pickle
import matplotlib.pyplot as plt

import root_numpy
from sklearn.metrics import roc_auc_score, roc_curve

from rep.estimators import TMVAClassifier

In [3]:
track = 'Long'
particle = 'Electron'

data_path = "/notebooks/data/MC2015Sim09Dev03/TrainMixture/TrainPhysTks-EvalPhysTks-NoReweight/\
GhostAccFrac1.0/TMVA-Run2-NoTkLikCD/" + track

work_path = "/notebooks/mikhail91/PID/mikhail_hushchyn/baseline/MC2015Sim09Dev03/TrainMixture/TrainPhysTks-EvalPhysTks-NoReweight/GhostAccFrac1.0/" + \
particle + "/" + track + "/TMVA/kMLP"

netconfig_path = "/notebooks/data/configs/networks/TMVA-Run2-NoTkLikCDVelodEdx/" + "GlobalPID_" \
+ particle + "_" + track + "_ANN.txt"





particle_pdg_codes = {"all": 999999,
                    "Ghost": 0,
                    "Electron": 11,
                    "Muon": 13,
                    "Pion": 211,
                    "Kaon": 321,
                    "Proton": 2212}

pdg = particle_pdg_codes[particle]





filename = data_path + "/data_train.csv"

n = sum(1 for line in open(filename)) - 1
s = n//10
skip = sorted(random.sample(xrange(1,n+1),n-s))

data = pandas.read_csv(filename, skiprows=skip)
data['TrackCloneDist'] *= 1 + 0.00001 * numpy.random.rand(len(data))

labels = (numpy.abs(data.MCParticleType.values) == pdg) * 1.





from sklearn.cross_validation import train_test_split

data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.3, random_state=42)







netconfig = numpy.loadtxt(netconfig_path, dtype='S', delimiter='\n', comments='!')
features = []
spectator_features = []

for var in netconfig[5:]:
    
    if var.find('#') == -1:
        features.append(var)
    else:
        spectator_features.append(var[1:])

In [3]:
data.shape

(1200000, 109)

In [9]:
import numpy
import root_numpy
data_train_root = numpy.array(data[features+['MCParticleType']].values.tolist())
data_train_root[:, :-1] += (0. + 0.00001*numpy.random.rand(data_train_root.shape[0], data_train_root.shape[1]-1))
data_train_root = data_train_root.view(dtype=zip(features+['MCParticleType'], [float]*(len(features) + 1)))
root_numpy.array2root(data_train_root, filename='data_train.root', treename='tree', mode='recreate')

In [10]:
data_train_root.shape

(1200, 1)

# Read data

In [24]:
import ROOT
f = ROOT.TFile('data_train.root')
ntuple = f.Get('tree')

In [54]:
a = ntuple.GetBranch('MCParticleType')
import root_numpy
a = root_numpy.root2array('data_train.root', treename='tree', branches=['MCParticleType'])

In [71]:
a = numpy.asarray(numpy.abs(a), dtype=float)

In [73]:
(a == 11).sum()

7

In [77]:
nTest_Signal = int(0.3 * (a == 11).sum())
nTest_Bkg = int(0.3 * (a != 11).sum())

print nTest_Signal, nTest_Bkg

2 357


# Train TMVA MLP

In [20]:
import ROOT
ROOT.TMVA.Tools.Instance()
fout = ROOT.TFile("test/test.root","RECREATE")

factory = ROOT.TMVA.Factory("TMVAClassification", fout,
                            ":".join([
                                "!V",
                                "!Silent",
                                "Color",
                                "DrawProgressBar",
                                "Transformations=I",#;D;P;G,D
                                "AnalysisType=Classification"]
                                     ))

for i in range(0, len(features)):
    factory.AddVariable(features[i],"F")
#factory.AddVariable("TrackPt","F")



factory.AddSignalTree(ntuple)
factory.AddBackgroundTree(ntuple)
 
# cuts defining the signal and background sample
sigCut = ROOT.TCut("abs(MCParticleType) == 11")
bgCut = ROOT.TCut("abs(MCParticleType) != 11")
 
factory.PrepareTrainingAndTestTree(sigCut,   # signal events
                                   bgCut,    # background events
                                   ":".join([
                                        "nTrain_Signal=0",
                                        "nTrain_Background=0",
                                        "SplitMode=Random",
                                        "NormMode=NumEvents",
                                        "!V"
                                       ]))

In [21]:
%%time

method = factory.BookMethod(ROOT.TMVA.Types.kMLP, "MLP",
                   ":".join([
                       "H",
                       "V",
                       "NCycles=50",
                       "HiddenLayers=10",
                       "EpochMonitoring=true",
                       "UseRegulator=true",
                       "ConvergenceImprove=1e-16",
                       "ConvergenceTests=15",
                       "VarTransform=Norm",
                       "NeuronType=sigmoid",
                        "TrainingMethod=BP",
                        "EstimatorType=CE"
                       ]))
 
factory.TrainAllMethods()
factory.TestAllMethods()
factory.EvaluateAllMethods()

CPU times: user 3.26 s, sys: 93 ms, total: 3.36 s
Wall time: 3.42 s


TDirectoryFile::Append:0: RuntimeWarning: Replacing existing TH1: convergencetest___mlp_train_epoch_0000_S (Potential memory leak).
TDirectoryFile::Append:0: RuntimeWarning: Replacing existing TH1: convergencetest___mlp_train_epoch_0000_B (Potential memory leak).
TDirectoryFile::Append:0: RuntimeWarning: Replacing existing TH1: epochmonitoring___epoch_0000_weights_hist01 (Potential memory leak).
TDirectoryFile::Append:0: RuntimeWarning: Replacing existing TH1: epochmonitoring___epoch_0000_weights_hist12 (Potential memory leak).
TDirectoryFile::Append:0: RuntimeWarning: Replacing existing TH1: convergencetest___mlp_test_epoch_0000_S (Potential memory leak).
TDirectoryFile::Append:0: RuntimeWarning: Replacing existing TH1: convergencetest___mlp_test_epoch_0000_B (Potential memory leak).


In [ ]:
import ROOT
reader = ROOT.TMVA.Reader()
import array

L = []

for i in range(0, len(features)):
    var = array.array('f',[0])
    reader.AddVariable(features[i],var)
    L.append(var)


reader.BookMVA("MLP","weights/TMVAClassification_MLP.weights.xml")

In [ ]:
test = numpy.array(data_test[features+['MCParticleType']].values.tolist())

In [ ]:
test_lab = (numpy.abs(test[:, -1]) == 11) * 1.

In [ ]:
probas = []
for i in range(0, len(test)):
    for k in range(0, len(L)):
        L[k][0] = test[i,:][k]
    probas.append(reader.EvaluateMVA("MLP"))


In [ ]:
probas = numpy.array(probas)
plt.hist(probas[test_lab==1], color='b', alpha=0.5, label='1', normed=True)
plt.hist(probas[test_lab==0], color='r', alpha=0.5, label='0', normed=True)
plt.legend(loc='best')
plt.show()

In [ ]:
roc_auc = roc_auc_score(test_lab, probas)
print roc_auc